In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("artworks.csv")

In [4]:
df.shape

(110618, 30)

In [5]:
df.head()

,_id,acquisition_year,acquisition_mode,date_creation,type,ensemble_id,domain,domain_leaf,domain_deno_for_grid,domain_description_mst,...,recap_description,all_realized_operations_history,hanging_history,hanging_history_m34,temporary_exhibitions_m30,exposition_out_of_folder,expositions,expositions_without_current,inscriptions,authors
0,150000000022692,1949.0,Achat,[avant 1949],individual,NaN,Peinture,Peinture,Peinture,Huile sur toile,...,NaN,2016/11/04 - 210I - Déplacement vers loc. inte...,NaN,Phantastische Kunst des XX Jahrhunderts : Bâle...,Phantastische Kunst des XX Jahrhunderts : Bâle...,Phantastische Kunst des XX Jahrhunderts : Bâle...,Phantastische Kunst des XX Jahrhunderts : Bâle...,Phantastische Kunst des XX Jahrhunderts : Bâle...,S.B.DR. : g. de chirico,6660500
1,150000001616310,NaN,Don,2003 - 2005,individual,NaN,Domaine non saisi,NaN,Domaine non saisi,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
2,150000000369068,2011.0,Achat,2004,individual,NaN,Objet/Design,Chaise,Objet/Design,"Chaise|Chaise de récupération, béquille en aci...",...,NaN,2013/07/17 - 210I - Déplacement vers loc. inte...,2013/07/02 - 1N4 : Niveau 4 Centre|Dossier n° ...,Présentation des collections permanentes (coll...,NaN,NaN,NaN,Présentation des collections permanentes (coll...,NaN,9000000000087765
3,150000000369087,2011.0,Achat,2009 - 2010,individual,NaN,Objet/Design,"Audio-visuel,informatique, comm. et repro.",Objet/Design,"série de 4 clés USB|Clés en métal, technologie...",...,NaN,2013/07/12 - 210I - Déplacement vers loc. inte...,2013/07/02 - 1N4 : Niveau 4 Centre|Dossier n° ...,Présentation des collections permanentes (coll...,NaN,NaN,NaN,Présentation des collections permanentes (coll...,NaN,9000000000087765
4,150000000454622,2011.0,Don,2009,individual,NaN,Objet/Design,Objet/Design,Objet/Design,"Matériel de récupération, hanse en métal, béton",...,NaN,2012/03/15 - 210I - Déplacement vers loc. inte...,NaN,NaN,NaN,NaN,NaN,NaN,,9000000000087765


## remove html tags

In [6]:
import re

def get_list_from_html(field):
    if(pd.isnull(field)):
        return []
    else:
        regex_item = re.compile('<li>(.*?)<\/li>', flags=re.S)
        tab = regex_item.split(str(field))
        new_tab = []
        for item in tab:
            if len(item) > 0 and 'ul>' not in item and item != '\n':
                new_tab.append(item)
        return new_tab

In [7]:
# df["operation_list"] = df["all_realized_operations_history"].apply(get_list_from_html)

In [8]:
df["operation_list"] = df["all_realized_operations_history"].apply(lambda x: str(x).split(";sep;"))

## remove no code cases from operation list

In [9]:
get_cleaned_list = lambda x : [i for i in x if len(i.split('-')) > 2]
remove_invalid_date = lambda x : [i for i in x if (int(i.split('-')[0][:4])>1678 and int(i.split('-')[0][:4])<2018)]
df["operation_list_clean"] = df["operation_list"].apply(get_cleaned_list).apply(remove_invalid_date)

In [10]:
# get_operation_between= lambda x : [i for i in x if (int(i.split('-')[0][:4])>1997 and int(i.split('-')[0][:4])<2001)]
# df["operation_list_1997_2000"] = df["operation_list_clean"].apply(get_operation_between)

## filter operation after 2001 && with code_loc only

In [11]:
code_loc = ["210I","211I","212I","213I","214I","216I","220I", "221I","241I","260I","261I","262I","299I","730I","750I","760I",\
           "210E","211E","220E","221E","230E","240E","241E","250E","260E","299E","242E", "244E","760E"]

In [12]:
# filter_operation = lambda x : [i for i in x if (int(i.split('-')[0][:4])>2000 and i.split(' - ')[1] in code_loc)]
filter_operation = lambda x : [i for i in x if (int(i.split('-')[0][:4])>2000) ]
df["operation_list_after_2001"] = df["operation_list_clean"].apply(filter_operation)

# select column

In [13]:
df_movement = df[["_id", "title_notice", "operation_list_after_2001"]]

In [14]:
df_movement.shape

(110618, 3)

In [15]:
df_movement["operation_times"] = df_movement["operation_list_after_2001"].apply(lambda x: len(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
df_movement = df_movement[df_movement.operation_times != 0]

In [17]:
df_movement.shape

(108081, 4)

In [18]:
sum(df_movement["operation_times"])

528711

## get opt field(date, code, opt_name, details), store into dict

In [29]:
def format_date(date):
    return date.replace("/00", "/01")

* Branche stockage : 
2AR
2B
2C
2DE
2F
2L
2M
2N
2P
2S
2V
2TR
* Branche ateliers : 
2EMB
2ENC
* Branche restauration : 
2R
2NR
2QUA
* Branche photo : 
2PHO
* Branche constats : 
2ALV 
2NALV
2CON

* Subbranch type storage : 
1B
1D
1E
1F
1I
1J
1K
1L
1M
1N3
1T
6B
* Subbranch type restoration :
1LTR
* Subbranch type photography / digitalization : 
1STUPHO
1SAV
1STUVID
* Subbranch type studios : 
1A
* Subbranch permanent exhibition spaces : 
1N4S
1N4R
1N4T
1N4E
1N4GALCONS
1N4GALPRO
1N5S
1N5R
1N5T
1N5E
* Subbranch temporary exhibition spaces :
1N0
1N1
1N4GAG
1N4GAL0
1N4GALMUS
1N6
1PIAZZA
1GALPHO
1GRANDESALLE

In [30]:
branches = {"PN_stockage": ["2A", "2B", "2C", "2DE", "2F", "2L", "2M", "2N", "2P", "2S", "2V", "2TR"],\
"PN_ateliers":["2EMB", "2ENC"],\
"PN_restauration":["2R", "2NR", "2QUA"],\
"PN_photo": ["2PHO"],\
"PN_constats": ["2ALV", "2NALV", "2CON"],\
"CPINTER_storage":["1B", "1D", "1E", "1F", "1I", "1J", "1K", "1L", "1M", "1N3", "1T", "6B"],\
"CPINTER_restoration":["1LTR"],\
"CPINTER_photo":["1STUPHO", "1SAV", "1STUVID"],\
"CPINTER_studio":["1A"],\
"CPPUB_perm": ["1N4S", "1N4R", "1N4T", "1N4E", "1N4GALCONS", "1N4GALPRO", "1N5S", "1N5R", "1N5T", "1N5E"],\
"CPPUB_tempo": ["1N0", "1N1", "1N4GAG", "1N4GAL0", "1N4GALMUS", "1N6", "1PIAZZA", "1GALPHO", "1GRANDESALLE"],\
}

In [31]:
external_branch = ["M20","M22","M25","M29","M32","M36","M99"]
deposit_branch = ["M10", "M11", "M19"]

In [32]:
def dict_mapper(pattern, dict_branch = branches):
    cand = []
    for cat, pat in dict_branch.items():
        if pattern in [pat_part[0:len(pattern)] for pat_part in pat]:
            cand += [cat]
    return cand

def get_branch(text, dict_branch = branches):
    branch_list = dict_mapper(text[0], dict_branch = dict_branch)
    if len(text) < 2 or branch_list == []:
        return "unknown"
    else:
        i = 1
        while(len(branch_list) > 1):
            i += 1
            if i>len(text):
                break
            branch_list = dict_mapper(text[0:i],dict_branch = dict_branch)
    if branch_list == []:
        j = len(text)
        while(len(branch_list) < 1):
            if j<0:
                return text
            j -= 1
            branch_list = dict_mapper(text[0:j],dict_branch = dict_branch)
    return branch_list[0]

In [33]:
def get_branch_field(opt):
    if(opt.split(' - ')[1] in code_loc):
        if("I" in opt.split(" - ")[1]):
            return get_branch(opt.split(" - ")[3])
        else:
            opt_detail = "-".join(opt.split(" - ")[3:])
            if(any(s in opt_detail for s in external_branch)):
                return "EXPO"
            elif(any(s in opt_detail for s in deposit_branch)):
                return "DEPOT"
            else:
                return "unknown"
    else:
        return "nonlocation"

In [34]:
get_opt_field = lambda x : sorted([{"date": format_date(i.split(' - ')[0]), "opt_code":i.split(" - ")[1],\
                             "opt_name": i.split(" - ")[2], "opt_branch": get_branch_field(i),\
                            "opt_detail": "-".join(i.split(" - ")[3:])}\
                            for i in x], key=lambda d: d['date'])

In [35]:
def get_opt_prop(x):
    return [{"date": format_date(i.split(' - ')[0]), "opt_code":i.split(" - ")[1],\
             "opt_name": i.split(" - ")[2], "opt_branch": get_branch_field(i),\
            "opt_detail": "-".join(i.split(" - ")[3:])}\
            for i in x]

In [36]:
df_movement["opt_field"] = df_movement["operation_list_after_2001"].apply(get_opt_field)

In [37]:
list(df_movement)

['_id',
 'title_notice',
 'operation_list_after_2001',
 'operation_times',
 'opt_field']

In [38]:
df_movement=df_movement.drop(["operation_times","operation_list_after_2001"], axis=1)

In [39]:
df_movement.to_json("artwork_operations_with_branch_complete.json", orient="records")

# clean date creation

In [5]:
import re
years_re = re.compile(r"\d{4}")
def clean_creation_date(creation_date):
    years = years_re.findall(str(creation_date))
    if len(years)>=1:
        return int(min(years))
    else:
        return None

df["date_creation"] = df["date_creation"].apply(clean_creation_date)